In [35]:
#####数据分析
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

train_data = pd.read_csv('data/train.csv')
test_data = pd.read_csv('data/test.csv')

print(train_data.shape[0])

1225029


In [3]:
# for i in train_data['landmark_id']:
#     if i == 14950:
#         print(i)
#         break
# ##0-14950


#train_data['landmark_id'][1225028]

count=0
for i in train_data['id']:
    count+=1    
print(count) ###1225029：0-1225028

# n=0
# for img_range_low in range(0,1181279,43751):
#     img_range_high=img_range_low+43751+1
#     for img_num in range(img_range_low,img_range_high,1):
#         a=train_data['id'][img_num]
#         n=img_range_low
# print(n)

1225029


In [33]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

train_data = pd.read_csv('labels_split/train-0.csv')
_id=train_data['id'][0]
print(_id+".jpg")

c90ca2e5e4e8e94c.jpg


In [ ]:
##############内存不够
import os
from PIL import Image
file_dir="data/train_img/"

images = []
labels = []
count=0
for img_name in train_data['id']:
    
    img_path=file_dir+img_name+".jpg"
    if os.path.exists(img_path):
        im = Image.open(img_path).convert('RGB') #转换为灰度图，选项:1,L,P,RGB,RGBA,CMYK,YCbCr,I,F
        imResize = im.resize((299, 299))
        imageData = np.array(imResize)
        images.append(imageData)
        #(filename,extension) = os.path.splitext(img_name)

        labels.append(train_data['landmark_id'][count])
    
    count+=1
        
# images_array = np.array(images)
# labels_array = np.array(labels)
X = np.array(images)
y = np.array(labels)

In [1]:
###################切分数据--1225029：   以及尝试训练
#####数据分析
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from PIL import Image

import tensorflow as tf 
from sklearn.model_selection import train_test_split #数据拆分
from keras.utils import np_utils #编码器
from keras.models import load_model

import keras.backend.tensorflow_backend as KTF  
KTF.set_session(tf.Session(config=tf.ConfigProto(device_count={'gpu':0}))) 

from inception_v4 import create_inception_v4

#model = create_inception_v4()
model = load_model('temp.h5')
model.compile(loss = 'categorical_crossentropy',optimizer='adam',metrics=['accuracy'])


train_data = pd.read_csv('data/train.csv')
test_data = pd.read_csv('data/test.csv')
file_dir="data/train_img/"

step=2612 ##18284,9142，4571
BATCH_SIZE=16 ##16

images = []
labels = []
count=0

#model = create_inception_v4()

for epoch_t in range(0,50):
    for img_range_low in range(0,1225029-step,step):
        img_range_high=img_range_low+step+1
        images = []
        labels = []

        for img_num in range(img_range_low,img_range_high,1):
            img_path=file_dir+train_data['id'][img_num]+".jpg"
            if os.path.exists(img_path):
                im = Image.open(img_path).convert('RGB') #转换为灰度图，选项:1,L,P,RGB,RGBA,CMYK,YCbCr,I,F
                imResize = im.resize((299, 299))
                imageData = np.array(imResize)
                images.append(imageData)
                #(filename,extension) = os.path.splitext(img_name)

                labels.append(train_data['landmark_id'][img_num])

        X = np.array(images)
        y = np.array(labels)
        print(X.shape)
        
        if(X.shape[0]>32):
            X = X.reshape(X.shape[0],299,299,3).astype('float32')
            X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.1,random_state=30)

            y_train = np_utils.to_categorical(y_train,14951) #矢量编码
            y_test = np_utils.to_categorical(y_test,14951)

            model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=1,batch_size=BATCH_SIZE,verbose=1)
    
        
# images_array = np.array(images)
# labels_array = np.array(labels)
# X = np.array(images)
# y = np.array(labels)

Using TensorFlow backend.
/usr/local/lib/python3.5/dist-packages/keras/engine/topology.py:1253: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  return cls(**config)


(2598, 299, 299, 3)
Train on 2338 samples, validate on 260 samples
Epoch 1/1
2338/2338 [==============================] - 163s 70ms/step - loss: 8.8489 - acc: 0.0449 - val_loss: 8.7548 - val_acc: 0.0346
(2603, 299, 299, 3)
Train on 2342 samples, validate on 261 samples
Epoch 1/1
2342/2342 [==============================] - 149s 63ms/step - loss: 8.7726 - acc: 0.0384 - val_loss: 8.8968 - val_acc: 0.0383
(2603, 299, 299, 3)
Train on 2342 samples, validate on 261 samples
Epoch 1/1
1712/2342 [====================>.........] - ETA: 38s - loss: 8.8143 - acc: 0.0356

KeyboardInterrupt: 

In [ ]:
###################按label划分train.csv
import csv
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

train_data = pd.read_csv('data/train.csv')

count=0
for i in train_data['id']:
    count+=1    
print(count) ###1225029：0-1225028

for i in range(0,count):
    _id=train_data['id'][i]
    _landId=train_data['landmark_id'][i]
    
    file_num = int(_landId/20)
    file_path="labels_split/train-%d.csv"%(file_num)
    
    if os.path.exists(file_path):
        da = [_id, _landId]
        # 写入数据
        csvFile = open(file_path, "a+")
        writer = csv.writer(csvFile)
        # 写入的内容都是以列表的形式传入函数
        writer.writerow(da)
        csvFile.close()
    else:
        fileHeader = ["id", "landmark_id"]
        # 假设我们要写入的是以下两行数据
        da = [_id, _landId]
        # 写入数据
        csvFile = open(file_path, "a+")
        writer = csv.writer(csvFile)
        # 写入的内容都是以列表的形式传入函数
        writer.writerow(fileHeader)
        writer.writerow(da)
        csvFile.close()

In [1]:
###################按label切分数据，每次训练20类：   以及尝试训练
#####数据分析
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from PIL import Image

import tensorflow as tf 
from sklearn.model_selection import train_test_split #数据拆分
from keras.utils import np_utils #编码器
from keras.models import load_model
from keras.optimizers import Adam

# import keras.backend.tensorflow_backend as KTF  
# KTF.set_session(tf.Session(config=tf.ConfigProto(device_count={'gpu':0}))) 

from inception_v4 import create_inception_v4

#model = create_inception_v4()
model = load_model('temp.h5')
#model.compile(loss = 'categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

#test_data = pd.read_csv('data/test.csv')
file_dir="data/train_img/"

BATCH_SIZE=16 ##16
lr_power = 0.9
lr_base=0.001

for epoch_time in range(0,20):
    lr_now = lr_base * ((1 - float(epoch_time)/20) ** lr_power)
    def_adam=Adam(lr=lr_now)
    model.compile(loss = 'categorical_crossentropy',optimizer=def_adam,metrics=['accuracy'])
    
    for file_num in range(77,748): #############14950/20  0-748,temp:32-748,77-,
        train_csv_path="labels_split/train-%d.csv"%(file_num)
        train_data = pd.read_csv(train_csv_path)
        
        total_num=train_data.shape[0]
        print("csv_num:%d"%(file_num))
        ifLarge=int(total_num/2500)
        if(ifLarge==0):
            images = []
            labels = []

            for img_num in range(0,total_num):
                img_path=file_dir+train_data['id'][img_num]+".jpg"
                if os.path.exists(img_path):
                    im = Image.open(img_path).convert('RGB') #转换为灰度图，选项:1,L,P,RGB,RGBA,CMYK,YCbCr,I,F
                    imResize = im.resize((299, 299))
                    imageData = np.array(imResize)
                    images.append(imageData)
                    #(filename,extension) = os.path.splitext(img_name)

                    labels.append(train_data['landmark_id'][img_num])


            X = np.array(images)
            y = np.array(labels)
            print(X.shape)

            if(X.shape[0]>1):
                X = X.reshape(X.shape[0],299,299,3).astype('float32')
                X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.1,random_state=30)

                y_train = np_utils.to_categorical(y_train,14951) #矢量编码
                y_test = np_utils.to_categorical(y_test,14951)

                model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=BATCH_SIZE,verbose=1)
        else:
            img_num_low=0
            img_num_high=0
            for img_num_low in range(0,total_num-2499,2500):
                img_num_high=img_num_low+2500
                images = []
                labels = []

                for img_num in range(img_num_low,img_num_high):
                    img_path=file_dir+train_data['id'][img_num]+".jpg"
                    if os.path.exists(img_path):
                        im = Image.open(img_path).convert('RGB') #转换为灰度图，选项:1,L,P,RGB,RGBA,CMYK,YCbCr,I,F
                        imResize = im.resize((299, 299))
                        imageData = np.array(imResize)
                        images.append(imageData)
                        #(filename,extension) = os.path.splitext(img_name)

                        labels.append(train_data['landmark_id'][img_num])

                X = np.array(images)
                y = np.array(labels)
                print(X.shape)

                if(X.shape[0]>1):
                    X = X.reshape(X.shape[0],299,299,3).astype('float32')
                    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.1,random_state=30)

                    y_train = np_utils.to_categorical(y_train,14951) #矢量编码
                    y_test = np_utils.to_categorical(y_test,14951)

                    model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=BATCH_SIZE,verbose=1)

            images = []
            labels = []  
            for img_num in range(img_num_high,total_num):
                img_path=file_dir+train_data['id'][img_num]+".jpg"
                if os.path.exists(img_path):
                    im = Image.open(img_path).convert('RGB') #转换为灰度图，选项:1,L,P,RGB,RGBA,CMYK,YCbCr,I,F
                    imResize = im.resize((299, 299))
                    imageData = np.array(imResize)
                    images.append(imageData)
                    #(filename,extension) = os.path.splitext(img_name)

                    labels.append(train_data['landmark_id'][img_num])

            X = np.array(images)
            y = np.array(labels)
            print(X.shape)

            if(X.shape[0]>1):
                X = X.reshape(X.shape[0],299,299,3).astype('float32')
                X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.1,random_state=30)

                y_train = np_utils.to_categorical(y_train,14951) #矢量编码
                y_test = np_utils.to_categorical(y_test,14951)

                model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=16,verbose=1)
                
    
        
# images_array = np.array(images)
# labels_array = np.array(labels)
# X = np.array(images)
# y = np.array(labels)

Using TensorFlow backend.
/usr/local/lib/python3.5/dist-packages/keras/engine/topology.py:1253: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  return cls(**config)


csv_num:35
(656, 299, 299, 3)
Train on 590 samples, validate on 66 samples
Epoch 1/10
590/590 [==============================] - 54s 92ms/step - loss: 15.5886 - acc: 0.0000e+00 - val_loss: 16.1181 - val_acc: 0.0000e+00
Epoch 2/10
590/590 [==============================] - 37s 63ms/step - loss: 13.3882 - acc: 0.0000e+00 - val_loss: 13.4595 - val_acc: 0.0000e+00
Epoch 3/10
590/590 [==============================] - 37s 63ms/step - loss: 11.5033 - acc: 0.0000e+00 - val_loss: 11.0913 - val_acc: 0.0000e+00
Epoch 4/10
590/590 [==============================] - 38s 65ms/step - loss: 10.5815 - acc: 0.0000e+00 - val_loss: 10.0759 - val_acc: 0.0000e+00
Epoch 5/10
590/590 [==============================] - 38s 65ms/step - loss: 9.8755 - acc: 0.0000e+00 - val_loss: 9.0815 - val_acc: 0.0000e+00
Epoch 6/10
590/590 [==============================] - 38s 65ms/step - loss: 9.0221 - acc: 0.0000e+00 - val_loss: 8.4870 - val_acc: 0.0000e+00
Epoch 7/10
590/590 [==============================] - 38s 65ms/st

559/559 [==============================] - 36s 64ms/step - loss: 8.7308 - acc: 0.0000e+00 - val_loss: 8.4614 - val_acc: 0.0000e+00
Epoch 4/10
559/559 [==============================] - 36s 64ms/step - loss: 7.4611 - acc: 0.0215 - val_loss: 6.9223 - val_acc: 0.0000e+00
Epoch 5/10
559/559 [==============================] - 36s 64ms/step - loss: 6.2415 - acc: 0.0429 - val_loss: 5.5870 - val_acc: 0.0000e+00
Epoch 6/10
559/559 [==============================] - 36s 64ms/step - loss: 5.4253 - acc: 0.0805 - val_loss: 4.3879 - val_acc: 0.0000e+00
Epoch 7/10
559/559 [==============================] - 36s 64ms/step - loss: 4.8552 - acc: 0.1216 - val_loss: 3.5099 - val_acc: 0.1905
Epoch 8/10
559/559 [==============================] - 36s 64ms/step - loss: 4.6325 - acc: 0.1181 - val_loss: 3.2892 - val_acc: 0.1746
Epoch 9/10
559/559 [==============================] - 36s 64ms/step - loss: 4.3661 - acc: 0.1163 - val_loss: 3.1380 - val_acc: 0.1746
Epoch 10/10
559/559 [==============================] 

Epoch 5/10
491/491 [==============================] - 32s 64ms/step - loss: 8.0292 - acc: 0.0244 - val_loss: 6.3139 - val_acc: 0.0000e+00
Epoch 6/10
491/491 [==============================] - 32s 64ms/step - loss: 6.7054 - acc: 0.0570 - val_loss: 4.9053 - val_acc: 0.0000e+00
Epoch 7/10
491/491 [==============================] - 32s 64ms/step - loss: 6.0129 - acc: 0.0876 - val_loss: 3.8641 - val_acc: 0.1636
Epoch 8/10
491/491 [==============================] - 32s 64ms/step - loss: 5.4828 - acc: 0.1079 - val_loss: 3.4471 - val_acc: 0.2909
Epoch 9/10
491/491 [==============================] - 32s 64ms/step - loss: 5.0252 - acc: 0.1365 - val_loss: 3.4651 - val_acc: 0.2545
Epoch 10/10
491/491 [==============================] - 32s 64ms/step - loss: 4.8230 - acc: 0.1446 - val_loss: 3.5566 - val_acc: 0.2545
csv_num:57
(1262, 299, 299, 3)
Train on 1135 samples, validate on 127 samples
Epoch 1/10
1135/1135 [==============================] - 73s 64ms/step - loss: 12.1413 - acc: 0.0000e+00 - val

343/343 [==============================] - 22s 65ms/step - loss: 9.0232 - acc: 0.0000e+00 - val_loss: 8.6536 - val_acc: 0.0000e+00
Epoch 8/10
343/343 [==============================] - 22s 65ms/step - loss: 8.5104 - acc: 0.0000e+00 - val_loss: 7.7487 - val_acc: 0.0000e+00
Epoch 9/10
343/343 [==============================] - 22s 65ms/step - loss: 8.0615 - acc: 0.0029 - val_loss: 7.0957 - val_acc: 0.0000e+00
Epoch 10/10
343/343 [==============================] - 22s 64ms/step - loss: 7.5770 - acc: 0.0146 - val_loss: 6.3255 - val_acc: 0.0000e+00
csv_num:67
(1787, 299, 299, 3)
Train on 1608 samples, validate on 179 samples
Epoch 1/10
1608/1608 [==============================] - 103s 64ms/step - loss: 11.4972 - acc: 0.0000e+00 - val_loss: 9.7393 - val_acc: 0.0000e+00
Epoch 2/10
1608/1608 [==============================] - 103s 64ms/step - loss: 9.0369 - acc: 6.2189e-04 - val_loss: 6.6602 - val_acc: 0.0000e+00
Epoch 3/10
1608/1608 [==============================] - 103s 64ms/step - loss: 7.

2202/2202 [==============================] - 142s 64ms/step - loss: 1.8708 - acc: 0.7984 - val_loss: 1.0451 - val_acc: 0.8163
Epoch 10/10
2202/2202 [==============================] - 142s 64ms/step - loss: 1.8369 - acc: 0.8052 - val_loss: 1.3126 - val_acc: 0.8082
(2207, 299, 299, 3)
Train on 1986 samples, validate on 221 samples
Epoch 1/10
1986/1986 [==============================] - 128s 64ms/step - loss: 1.9356 - acc: 0.7860 - val_loss: 1.8415 - val_acc: 0.7466
Epoch 2/10
1986/1986 [==============================] - 128s 64ms/step - loss: 1.9303 - acc: 0.7875 - val_loss: 1.8387 - val_acc: 0.7466
Epoch 3/10
1986/1986 [==============================] - 128s 64ms/step - loss: 1.9060 - acc: 0.7905 - val_loss: 2.1232 - val_acc: 0.7330
Epoch 4/10
1986/1986 [==============================] - 128s 64ms/step - loss: 1.9267 - acc: 0.7815 - val_loss: 3.4020 - val_acc: 0.6606
Epoch 5/10
1986/1986 [==============================] - 128s 64ms/step - loss: 1.9246 - acc: 0.7805 - val_loss: 2.3990 - 

KeyboardInterrupt: 

In [2]:
# #testing
# scores=model.evaluate(X_test,y_test,verbose=1)
# print('Test score:', scores[0])
# print('Test accuracy:', scores[1])

# #model.save('edge_model2_random.h5')
#model.save('temp.h5')

In [9]:
for i in range(1,1225028):
    if(1225028%i==0):
        print(i)

1
2
4
7
14
28
67
134
268
469
653
938
1306
1876
2612
4571
9142
18284
43751
87502
175004
306257
612514


In [1]:
from inception_v4 import create_inception_v4

model = create_inception_v4()

Using TensorFlow backend.
/notebooks/GoogleLandmarkRec/inception_v4.py:25: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), strides=(2, 2), padding="valid", use_bias=False)`
  x = Convolution2D(nb_filter, nb_row, nb_col, subsample=subsample, border_mode=border_mode, bias=bias)(x)
/notebooks/GoogleLandmarkRec/inception_v4.py:25: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), strides=(1, 1), padding="valid", use_bias=False)`
  x = Convolution2D(nb_filter, nb_row, nb_col, subsample=subsample, border_mode=border_mode, bias=bias)(x)
/notebooks/GoogleLandmarkRec/inception_v4.py:25: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), strides=(1, 1), padding="same", use_bias=False)`
  x = Convolution2D(nb_filter, nb_row, nb_col, subsample=subsample, border_mode=border_mode, bias=bias)(x)
/notebooks/GoogleLandmarkRec/inception_v4.py:42: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPo

/notebooks/GoogleLandmarkRec/inception_v4.py:25: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(256, (1, 7), strides=(1, 1), padding="same", use_bias=False)`
  x = Convolution2D(nb_filter, nb_row, nb_col, subsample=subsample, border_mode=border_mode, bias=bias)(x)
/notebooks/GoogleLandmarkRec/inception_v4.py:104: UserWarning: Update your `AveragePooling2D` call to the Keras 2 API: `AveragePooling2D((3, 3), strides=(1, 1), padding="same")`
  b4 = AveragePooling2D((3, 3), strides=(1, 1), border_mode='same')(input)
/notebooks/GoogleLandmarkRec/inception_v4.py:25: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (1, 1), strides=(1, 1), padding="same", use_bias=False)`
  x = Convolution2D(nb_filter, nb_row, nb_col, subsample=subsample, border_mode=border_mode, bias=bias)(x)
/notebooks/GoogleLandmarkRec/inception_v4.py:107: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`

Model weights loaded.


/notebooks/GoogleLandmarkRec/inception_v4.py:218: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=14951, activation="softmax")`
  out = Dense(output_dim=nb_classes, activation='softmax')(x)


In [2]:
model.compile(loss = 'categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [4]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from PIL import Image
import numpy as np
from keras.utils import np_utils #编码器


train_data = pd.read_csv('data/train.csv')
test_data = pd.read_csv('data/test.csv')
file_dir="data/train_img/"

labels =[]
for img_num in range(0,100,1):
    img_path=file_dir+train_data['id'][img_num]+".jpg"
    if os.path.exists(img_path):
        im = Image.open(img_path).convert('RGB') #转换为灰度图，选项:1,L,P,RGB,RGBA,CMYK,YCbCr,I,F
        #imResize = im.resize((299, 299))
        #imageData = np.array(imResize)
        #images.append(imageData)
        #(filename,extension) = os.path.splitext(img_name)

        labels.append(train_data['landmark_id'][img_num])

#X = np.array(images)
y = np.array(labels)

y_test = np_utils.to_categorical(y,14951)

Using TensorFlow backend.


In [5]:
y

array([ 4676,  6651, 11284,  8429,  6231, 10400,  9779, 11288, 13170,
        6051,  8302,  3903, 11702,  6347,   691,  9633,  9633,  9633,
        7794,  9779,  4645,  2743,  4135,  6051,  6051,  2949,  4981,
        2427,  3457,  7615,  6051,  6707,  3724,  4987,  9633,  5140,
        7975,  9633,  5023, 13341,  9633,  9633, 11541,  2272,  5762,
        7058,  2743, 11254,  9633,  8395,  2743,  5985,  6685,  9633,
        1966, 12210,  4793,   369,  8692,  3878, 12156,  2743, 12712,
        3040,  6696,  7147,  2339,  7612,  9779,  3938,  4190, 12896,
        6051,  7416,  2338,  6309, 11255, 10900,  7058,  9633, 13797,
        2664,   148, 11247,  4352,  5554,  6523, 10600,  7033,  9633,
        6051,  7790, 12336,  8896, 10025,  6956,   452,  1038, 11437])

In [9]:
y_test[1][6651]

1.0